# Training Pipeline
* Create and return Feature View and its dataset
* Define, tune and train the regression model
* Evaluate model
* Push model to Hopsworks

In [ ]:
import os
import joblib
import pandas as pd
import hopsworks
from xgboost import XGBRegressor
from xgboost import plot_importance
import pickle
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score, f1_score, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from hsml.schema import Schema
from hsml.model_schema import ModelSchema
import numpy as np

import sys
sys.path.append('..')  # Add the parent directory (project root) to the Python path
from config import *

# Disable annoying warnings
import warnings
warnings.filterwarnings("ignore")

# Set path to save model and plots
if os.path.isdir(MODEL_PATH) == False:
    os.mkdir(MODEL_PATH)

# Create and Return Feature View in Hopsworks

In [ ]:
# Login to Hopsworks
project = hopsworks.login()

# Retrieve Feature Store
fs = project.get_feature_store()

In [ ]:
# Retrieve Feature Group
historical_weather_fg = fs.get_or_create_feature_group(
    name=FG_HISTORY_NAME,
    version=FG_HISTORY_V,
)

# Query: transform feature group into feature view. no feature groups to join
query = historical_weather_fg.select_all()

# Create Feature View in Hopsworks
feature_view = fs.create_feature_view(
    name=FEATURE_VIEW_NAME,
    version=FEATURE_VIEW_V,
    query=query,
    labels=['weather_code'],
)

# Create Training and Test set

Create training and test set split

In [ ]:
X_train, y_train, X_test, y_test = feature_view.train_test_split(test_size=0.2)

In [ ]:
print('Training set contains', X_train.shape[0], 'entries')
print('Training set contains', X_test.shape[0], 'entries')

# Model hyperameter tuning

In [ ]:
# Create model
xgb_model_tuning = XGBRegressor(objective='reg:squarederror') # Square error is the most common loss function for regression prediction problems

# Define the hyperparameter distributions for the random search
param_dist = {
    'learning_rate': LEARNING_RATE_RANGE,
    'n_estimators': N_ESTIMATORS_RANGE,
    'max_depth': MAX_DEPTH_RANGE,
    'subsample': SUBSAMPLE_RANGE,
    'colsample_bytree': COLSAMPLE_BYTREE_RANGE,
}

# Perform the tuning with random search and cross-validation
random_search = RandomizedSearchCV(estimator=xgb_model_tuning,
                                   param_distributions=param_dist,
                                   n_iter=10,
                                   scoring='neg_mean_squared_error',
                                   cv=N_FOLD_CV,
                                   random_state=42)

# fit data
random_search.fit(X_train, y_train)

# Get best parameters
best_params = random_search.best_params_
print("Best Hyperparameters:", best_params)

# Model Training

Train tuned model on the training set

In [ ]:
# Set tuned parameters
xgb_model = XGBRegressor(objective='reg:squarederror', **best_params)

# Train the model
xgb_model.fit(X_train, y_train)

In [ ]:
xgb_model.fit(X_train, y_train)

# Model evaluation

Predict on the unseen test set

In [ ]:
y_pred = xgb_model.predict(X_test)

Compute evaluation metrics

In [ ]:
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred, squared=False)
rmse = mean_squared_error(y_test, y_pred, squared=True)

# Round predicted value to closest weather code as a classification
y_pred_classified = np.round(y_pred).astype(int)

# Weighted-averaged F1 score
f1 = f1_score(y_test, y_pred_classified, average='weighted')

In [ ]:
print("R2: {:.2f}".format(r2))
print("MSE: {:.2f}".format(mse))
print("RMSE: {:.2f}".format(rmse))
print("F1 score: {:.2f}".format(f1))

#### F1 Score in-depth analysis

In [ ]:
# Plot F1 report
report = classification_report(y_test, y_pred_classified, output_dict=True)

# Extract the relevant metrics for each class
classes = [int(c) for c in report.keys() if c.isdigit()]  # Extract numeric classes
tp = [report[str(c)]['precision'] * report[str(c)]['support'] for c in classes]
fp = [(1 - report[str(c)]['precision']) * report[str(c)]['support'] for c in classes]
tn = [report[str(c)]['recall'] * report[str(c)]['support'] for c in classes]
fn = [(1 - report[str(c)]['recall']) * report[str(c)]['support'] for c in classes]

# Create a stacked bar plot
fig, ax = plt.subplots()
ax.bar(classes, tp, label='True Positives', color='green')
ax.bar(classes, fp, bottom=tp, label='False Positives', color='red')
ax.bar(classes, tn, bottom=np.array(tp) + np.array(fp), label='True Negatives', color='blue')
ax.bar(classes, fn, bottom=np.array(tp) + np.array(fp) + np.array(tn), label='False Negatives', color='orange')

# Add labels and title
plt.xlabel('Weather Code')
plt.ylabel('Count')
plt.title('F1 Metrics for Each Weather Code')

# Move the legend outside the plot using bbox_to_anchor
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

plt.savefig(MODEL_PATH + "/plot_f1.png")

# Show the plot
plt.show()

plt_f1 = plt

#### Distribution overview

In [ ]:
# Set the width of the bars
bar_width = 0.35

# Create an array for the x-axis positions
indices = np.arange(1, 11)

# Create bar plots for true labels and rounded predictions side by side
plt.bar(indices - bar_width/2, np.histogram(y_test, bins=np.arange(1, 12) - 0.5)[0], bar_width, label='True Labels', color='blue', edgecolor='black')
plt.bar(indices + bar_width/2, np.histogram(y_pred_classified, bins=np.arange(1, 12) - 0.5)[0], bar_width, label='Predictions', color='orange', edgecolor='black')

# Add labels and title
plt.xlabel('Weather Code')
plt.ylabel('Frequency')
plt.title('Frequency Distribution of Weather Codes')
plt.xticks(indices)
plt.legend()

# Show the plot
plt.show()

### Residuals

In [ ]:
# Differences between the observed (actual) values and the predicted values
# Ideally, the residuals should be randomly distributed around zero, indicating that the model's predictions are unbiased
# On the x axis I see the different weather codes (difficult to inspect here, but the goal is the distribution)

df_ = pd.DataFrame({
    "y_true": y_test,
    "y_pred": y_pred
})
residplot = sns.residplot(data=df_, x="y_true", y="y_pred", color='orange')
plt.title('Model Residuals')
plt.xlabel('Obsevation #')
plt.ylabel('Error')

plt.show()
fig = residplot.get_figure()
fig.show()

# Save residuals plot
fig.savefig(MODEL_PATH + "/plot_residuals.png")

### Feature importance

In [ ]:
# Scores for each feature based on how frequently they are used in the model during the training process,
# and how much they contribute to reducing the loss function

plot_importance(xgb_model)

# Model Registry

In [ ]:
# Retrieve model registry
mr = project.get_model_registry()

### Model Schema

In [ ]:
# Set up the Model Schema, which describes the inputs and outputs for a model
input_schema = Schema(X)
output_schema = Schema(y)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

### Save model locally

In [ ]:
# Save regressor model
joblib.dump(xgb_model, MODEL_PATH + '/'+ MODEL_NAME + '.pkl')

# Save F1 report
with open(MODEL_PATH + "/f1_report.txt", 'w') as file:
    file.write(f1_report)

### Upload model to Hopsworks

In [ ]:
# Define model for Hopsworks
weather_code_model = mr.python.create_model(
    name=MODEL_NAME, 
    metrics={
        'F1': f1,
        'R2': r2,
        'MSE': mse,
        'RMSE': rmse,
    },
    model_schema=model_schema, # attach model schema
    input_example=X_test.sample().values, 
    description="Weather Code predictor.")

# Upload model to Hopsworks
weather_code_model.save(MODEL_PATH)